<a href="https://colab.research.google.com/github/wiktoriaslomiany/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from ast import literal_eval
from tqdm import tqdm_notebook


In [8]:
import eli5

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
from eli5.sklearn import PermutationImportance

In [10]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [11]:
ls data


men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [16]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [17]:
test = {'key':'value'}
test['key']

str(test)

"{'key': 'value'}"

In [18]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features =  literal_eval(x.replace('\\"','"'))
  for item in features: 
    key = item['key'].lower().strip()
    value = item ['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [20]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [21]:
def get_name_feat(key):
  return 'feat_'+ key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [22]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_country of manufacture', 'feat_manufacturer',
       'feat_fabric care instructions', 'feat_resizable',
       'feat_cleaning, care & maintenance', 'feat_euro size', 'feat_parts',
       'feat_dial markers', 'feat_bracelet style', 'feat_clothing category'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False== df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [28]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [40]:
df ['brand'] = df ['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [43]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model('brand_cat', model)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Expected 2D array, got 1D array instead:
array=[143. 143. 143. ... 399. 399. 399.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Expected 2D array, got 1D array instead:
array=[  0.   0.   1. ... 399. 399. 399.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_sel

(nan, nan)

In [66]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_fabrication',
 'feat_location - country',
 'feat_shoe category',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_location - city/state',
 'feat_multi pack indicator',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_shoe closure_cat',
 'feat_place of origin_cat',
 'feat_compass_cat',
 'feat_rise_cat',
 'feat_combined shipping_cat',
 'feat_machine washable_cat',
 'feat_shoe width_cat',
 'feat_display technology_cat',
 'feat_fabric type_cat',
 'feat_gemstone type_cat',
 'feat_bridge/temple size:_cat',
 'feat_crystal_cat',
 'feat_sizearm_cat',
 'feat_case finish_cat',
 'feat_hairstyle_cat',
 'feat_boot height_cat',
 'feat_capacity_cat',
 'feat_item type_cat',
 'feat_is recyclable_cat',
 'feat_manuf

In [0]:
feats = ['brand_cat','feat_metal type_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat', 'feat_shape_cat' ]

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [80]:
x = df[ feats ].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(x,y);
eli5.show_weights(perm, feature_names=feats)

(-57.25834623921037, 4.242488386553487)


Weight,Feature
0.2549 ± 0.0080,brand_cat
0.1031 ± 0.0014,feat_material_cat
0.0252 ± 0.0022,feat_gender_cat
0.0178 ± 0.0025,feat_brand_cat
0.0128 ± 0.0003,feat_shape_cat
0.0091 ± 0.0014,feat_metal type_cat
0.0034 ± 0.0008,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [61]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{},
       {'style': 'n/a', 'gender': 'unisex', 'lens technology': 'no,62,6.75', 'assembled product dimensions (l x w x h)': '1.00 x 1.00 x 1.00 inches', 'fabric content': '100% n/a', 'color': 'blue', 'model': 'nikesun-ev0819-479', 'shipping weight (in pounds)': '1.0', 'mpn': 'ev0819-479', 'brand': 'nike', 'manufacturer_part_number': 'ev0819-479', 'age group': 'child'},
       {'style': 'athletic sneakers', 'condition': 'new with box', 'shade': 'white-hyper punch-black', 'type 2': 'tennis'},
       {'sport': 'soccer', 'condition': 'new without box', 'type': 'cleats'},
       {'style': 'running/cross training', 'material': 'synthetic/mesh', 'condition': 'new with box'}],
      dtype=object)

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
youth                  2
men||women             2
women ,�� unisex       2
women                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64